<font size="+3"><strong>3.5. Air Quality in Dar es Salaam 🇹🇿</strong></font>

In [1]:
import warnings

import wqet_grader

warnings.simplefilter(action="ignore", category=FutureWarning)
wqet_grader.init("Project 3 Assessment")

Exception: Could not connect to Grading Service API: HTTPConnectionPool(host='localhost', port=2400): Max retries exceeded with url: /1/track (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x282fcaad0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
import inspect
import time
import warnings
from pprint import PrettyPrinter
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns
from IPython.display import VimeoVideo
from pymongo import MongoClient
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import time
warnings.filterwarnings("ignore")

In [ ]:
pp = PrettyPrinter(indent=2)

In [ ]:
client = MongoClient(host='localhost', port=27017)

In [ ]:
pp.pprint(list(client.list_databases()))

In [ ]:
db = client["air-quality"]

In [ ]:
for c in db.list_collections():
    print(c['name'])

In [ ]:
dar = db['dar-es-salaam']

## Explore

In [ ]:
dar.count_documents({})

In [ ]:
result = dar.find_one({})
pp.pprint(result)

In [ ]:
sites = dar.distinct('metadata.site')
sites

In [ ]:
wqet_grader.grade("Project 3 Assessment", "Task 3.5.2", sites)

In [ ]:
print("Documents from site 23:", dar.count_documents({'metadata.site': 23}))
print("Documents from site 11:", dar.count_documents({'metadata.site': 11}))

In [ ]:
result = dar.aggregate(
    [
        
        {'$group': {'_id': '$metadata.site', 'count': {'$count': {}}}}
    ]
)
pp.pprint(list(result))

In [ ]:
readings_per_site = dar.aggregate(
    [
        {"$match": {"metadata.measurement": "P2"}},
        {'$group': {'_id': '$metadata.site', 'count': {'$count': {}}}}
    ]
)
pp.pprint(list(readings_per_site))

In [ ]:
def wrangle(collection, resample_rule="1H"):

    results = collection.find(
        {"metadata.site": 29, "metadata.measurement": "P2"},
        projection={"P2": 1, "timestamp": 1, "_id": 0},
    )

    # Read results into DataFrame
    df = pd.DataFrame(list(results)).set_index("timestamp")

    # Localize timezone
    df.index = df.index.tz_localize("UTC").tz_convert("Africa/Dar_es_Salaam")

    # Remove outliers
    df = df[df["P2"] < 500]

    # Resample and forward-fill
    y = df["P2"].resample(resample_rule).mean().fillna(method="ffill")

    return y

In [ ]:
y = wrangle(dar)
y.head()

## Explore Some More

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
y.plot(ax=ax, xlabel="Time", ylabel="PM2.5");
# Don't delete the code below 👇
plt.savefig("images/3-5-5.png", dpi=150)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Don't delete the code below 👇
y.rolling(168).mean().plot(ax=ax, ylabel="PM2.5", title="Weekly Rolling Averages");
plt.savefig("images/3-5-6.png", dpi=150)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Don't delete the code below 👇
plot_acf(y, ax=ax);
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient")
plt.savefig("images/3-5-7.png", dpi=150)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Don't delete the code below 👇
plot_pacf(y, ax=ax);
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient")
plt.savefig("images/3-5-8.png", dpi=150)

## Split

In [ ]:
cutoff_test = int(0.90*len(y))
y_train = y.iloc[:cutoff_test]
y_test = y.iloc[cutoff_test:]
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# Build Model

## Baseline

In [ ]:
y_train_mean = y_train.mean()
y_pred_baseline = [y_train_mean] * len(y_train)
mae_baseline = mean_absolute_error(y_train, y_pred_baseline)

print("Mean P2 Reading:", y_train_mean)
print("Baseline MAE:", mae_baseline)

## Iterate

In [ ]:
p_params = range(1, 31)
maes = []
for p in p_params:
    start_time = time.time()
    model = AutoReg(y_train, lags=p).fit()
    y_pred = model.predict().dropna()
    training_mae = mean_absolute_error(y_train[p:], y_pred)
    maes.append(training_mae)
    elapsed_time = round(time.time() - start_time, 2)
    print(f"Trained AutoReg {p} in {elapsed_time} seconds.")
mae_series = pd.Series(maes, name="mae", index=p_params)
mae_series.head()

In [ ]:
plt.plot(mae_series)

In [ ]:
best_p = p_params[mae_series.argmin()]
best_model = AutoReg(y_train, lags=best_p).fit()

In [ ]:
y_train_resid = model.resid
y_train_resid.name = "residuals"
y_train_resid.head()

In [ ]:
# Plot histogram of residuals

# Don't delete the code below 👇
plt.hist(y_train_resid)
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.savefig("images/3-5-14.png", dpi=150)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Don't delete the code below 👇
plot_acf(y_train_resid, ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient")
plt.title("Dar es Salaam, Training Residuals ACF")
plt.savefig("images/3-5-15.png", dpi=150)

## Evaluate

In [ ]:
y_pred_wfv = pd.Series()
history = y_train.copy()
for i in range(len(y_test)):
    AutoReg(history, lags=best_p).fit()
    next_pred = model.forecast()
    y_pred_wfv = y_pred_wfv.append(next_pred)
    history = history.append(y_test[next_pred.index])
y_pred_wfv.name = "prediction"
y_pred_wfv.index.name = "timestamp"
y_pred_wfv.head()

# Communicate Results

In [ ]:
y_test

In [ ]:
y_pred_wfv.values

In [ ]:
df_pred_test = pd.DataFrame(
    {
        "y_test": y_test.values,
        "y_pred_wfv": y_pred_wfv.values
        
    }
)
df_pred_test.head()
fig = px.line(df_pred_test, labels={"value": "PM2.5"})
fig.update_layout(
    title="Dar es Salaam, WFV Predictions",
    xaxis_title="Date",
    yaxis_title="PM2.5 Level",
)
# Don't delete the code below 👇
fig.write_image("images/3-5-18.png", scale=1, height=500, width=700)

fig.show()